In [1]:
from time import time
from uuid import uuid4

from faker import Faker
from vertica_python import connect, Connection
from vertica_python.vertica.cursor import Cursor

In [2]:
fake: Faker = Faker()

CONNECTION_INFO: dict = {
    'host': '127.0.0.1',
    'port': 5433,
    'user': 'dbadmin',
    'password': '',
    'database': 'docker',
    'autocommit': True,
}
connection: Connection = connect(**CONNECTION_INFO)

In [3]:
cursor: Cursor = connection.cursor()
cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS test_user_progress
    (
        user_id VARCHAR,
        film_id VARCHAR,
        viewed_frame INTEGER,
        ts DATETIME
    )
    """
)

In [4]:
BATCH_SIZE: int = 10000
BATCHES: int = 1000
TOTAL_RECORDS: float = BATCH_SIZE * BATCHES

In [5]:
start_time: float = time()

for batch in range(BATCHES):
    data = [
        (uuid4(), uuid4(), fake.random_int(min=0, max=1000), fake.date_time_between(start_date='-1y', end_date='now')) for _ in range(BATCH_SIZE)
    ]
    cursor.executemany(
            'INSERT INTO test_user_progress (user_id, film_id, viewed_frame, ts) VALUES (%s, %s, %s, %s)', data
        )

insertion_time: float = time() - start_time

insertion_speed: float = round(TOTAL_RECORDS / insertion_time, 2)

print('Insertion speed: {:,} records/sec'.format(insertion_speed))

Insertion speed: 40,694.49 records/sec


In [6]:
start_time: float = time()

cursor.execute(
    """
    SELECT
        user_id,
        sum(viewed_frame),
        max(viewed_frame)
    FROM test_user_progress
    WHERE ts > '2022-12-01 00:00:00'
    GROUP by user_id
    """
)

reading_time: float = time() - start_time

reading_speed: float = round(TOTAL_RECORDS / reading_time, 2)

print('Reading speed: {:,} records/sec'.format(reading_speed))

Reading speed: 1,011,023.92 records/sec


In [7]:
cursor.execute('DROP TABLE IF EXISTS test_user_progress')

cursor.close()
connection.close()
